<a href="https://colab.research.google.com/github/vuhpham94/nflx-adv-data-hw/blob/main/11052021/01-SkillDrill/PeterPan-VP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# remove punctuation, spaces and convert words to lowercase
import re
def clean_text(text):
       return re.sub(re.compile(r'[^a-zA-Z0-9\s]'),"",text).lower().strip()

In [ ]:
 #import packages
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# Read in data
from pyspark import SparkFiles

peterpan_url = "https://raw.githubusercontent.com/vuhpham94/nflx-adv-data-hw/main/11052021/01-SkillDrill/Resources/peterpan.txt"

sc = spark.sparkContext
sc.addFile(peterpan_url)

text_file = sc.textFile(SparkFiles.get("peterpan.txt"))
words = text_file.flatMap(lambda line: line.split(" "))

word_counts = words.map(clean_text)\
                   .map(lambda word: (word, 1))\
                   .reduceByKey(lambda a, b: a + b)
word_counts.take(20)

In [ ]:
# save word_counts RDD to file
word_counts.saveAsTextFile("./peterpan_word_counts")